In [2]:
import pandas as pd
import numpy as np
import time_series_transform.base as base
from time_series_transform.time_series_transformer import *

In [3]:
df = pd.read_csv('./Data/Train/sales_train_validation.csv')
df.index = df.id
df = df.drop('id',axis=1)

In [4]:
CATEGORICAL_DIM = [
    'item_id','dept_id','cat_id','store_id','state_id'
]
WINDOW_SIZE = 20
TEST_RANGE = WINDOW_SIZE + 20

In [5]:
testDate = list(map(lambda x: 'd_'+str(x),list(range(1913-TEST_RANGE,1914))))
trainDateDrop = list(map(lambda x: 'd_'+str(x),list(range(1,1600))))
test = df[CATEGORICAL_DIM+testDate]
train = df.drop(testDate+trainDateDrop,axis = 1)
SEQ_TRANSFORMER = []

In [6]:
T_SEQUENCE_SIZE = len(train.drop(CATEGORICAL_DIM,axis =1).columns)-WINDOW_SIZE +1

In [13]:
ptt = Pandas_Time_Series_Dataset(train)
for i in CATEGORICAL_DIM:
    ptt.set_config(name = i,
                   colNames = [i],
                   tensorType = 'category',
                   param = {"windowSize":0,"batchSize":T_SEQUENCE_SIZE,"outType":np.float32},
                   sequence_stack = None,
                   isResponseVar = False
                  )

In [14]:
gen = ptt.make_data_generator()

In [16]:
pass

In [12]:
ptt.config

{'item_id': {'colNames': 'item_id',
  'tensorType': 'category',
  'param': {'windowSize': 0, 'batchSize': 254, 'outType': numpy.float32},
  'sequence_stack': None,
  'responseVariable': False},
 'dept_id': {'colNames': 'dept_id',
  'tensorType': 'category',
  'param': {'windowSize': 0, 'batchSize': 254, 'outType': numpy.float32},
  'sequence_stack': None,
  'responseVariable': False},
 'cat_id': {'colNames': 'cat_id',
  'tensorType': 'category',
  'param': {'windowSize': 0, 'batchSize': 254, 'outType': numpy.float32},
  'sequence_stack': None,
  'responseVariable': False},
 'store_id': {'colNames': 'store_id',
  'tensorType': 'category',
  'param': {'windowSize': 0, 'batchSize': 254, 'outType': numpy.float32},
  'sequence_stack': None,
  'responseVariable': False},
 'state_id': {'colNames': 'state_id',
  'tensorType': 'category',
  'param': {'windowSize': 0, 'batchSize': 254, 'outType': numpy.float32},
  'sequence_stack': None,
  'responseVariable': False}}